# WEB SCRAPPING FACEBOOK POSTS IN A GROUP

- pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from dotenv import load_dotenv, dotenv_values 
import re
import os

### Configurar driver
En mi caso yo uso mozilla, así que para hacer scrapping con Selenium me es necesario descargar primero un driver como geckodriver para trabajar con Firefox.
En el caso de chrome creo que no es necesario, solo pueden llamar a chrome y ya como se ve abajo

In [ ]:
# #@title configurar driver para mozilla

# gecko_path = r'' #ponen la ruta del driver
# options = webdriver.FirefoxOptions()
# # options.add_argument("--headless") 
# #Para no tener la interfaz grafica

# service = Service(gecko_path)
# driver = webdriver.Firefox(service=service, options=options)

In [2]:
#@title configurar driver para chrome
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  
#Para no tener la interfaz grafica
driver = webdriver.Chrome(options=options)

Tienen que declarar las credenciales de la cuenta con la que entrarán a facebook

In [3]:
load_dotenv() #cargar el archivo .env
email_value= os.getenv('email') #sus datos
pass_value= os.getenv('pass')
if not email_value or not pass_value:
    raise ValueError("Please set 'email' and 'pass' in your .env file.")

In [4]:
driver.get("https://www.facebook.com")

#cargar credenciales
emailelement = driver.find_element(By.XPATH,'//*[@id="email"]')
emailelement.send_keys(email_value)
passelement = driver.find_element(By.XPATH,'//*[@id="pass"]')
passelement.send_keys(pass_value)
loginelement = driver.find_element(By.NAME, "login")
loginelement.click()
sleep(2)

Esta es la sección en que obtenemos los posts de x grupos de facebook. 
Es necesario hacer scrolls en la página, pues determinan cuántos posts se cargan en el html.

In [ ]:

groups_links_list = [
"https://www.facebook.com/groups/973136421423238/"
       #" https://www.facebook.com/groups/973136421423238/", "https://www.facebook.com/groups/1287841212267472/", "https://www.facebook.com/groups/1736164896891333"
    ] #acá pegué todos los grupos de facebook uis que encontré


for group in groups_links_list:
    driver.get(group)
    sleep(2)    
    body = driver.find_element(By.TAG_NAME, 'body')
    for _ in range(15):  #número de scrolls que se hacen por página
        body.send_keys(Keys.PAGE_DOWN)
        sleep(3)

    #encontrar spans que contienen posts, si le dan a inspeccionar pag en cada post, estos se encuentran contenidos en un span
    #la clase de estos es span es class="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h"
    contenedores = driver.find_elements(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u.x1yc453h")

    #recorrer cada div y extraer el texto ubicado en dir="auto" style="text-align: start;"
    for contenedor in contenedores:

        try: #el botón ver más tiene la clase class="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz xkrqix3 x1sur9pj xzsf02u x1s688f"
            ver_mas_btn = contenedor.find_element(By.CSS_SELECTOR, "div.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr") #hay que darle click a este para poder ver la totalidad de los mensajes
            ver_mas_btn.click()
            sleep(2)

            #extrae el texto completo después de hacer clic
            textos_expandidos = contenedor.find_elements(By.CSS_SELECTOR, "input[dir='auto'][style='text-align: start;']")
            for texto in textos_expandidos:
                print(texto.text)  #print del texto después de hacer clic
        except:
            textos = contenedor.find_elements(By.CSS_SELECTOR, "div[dir='auto'][style='text-align: start;']")
            for texto in textos:
                print(texto.text)  #print del texto en caso de no haber una opcion de ver mas

Cursos virtuales cortos y gratuitos SENA 2025. Para fortalecer tu perfil profesional.


In [ ]:
from datetime import datetime

def extract_posts_safely(driver, group_url, num_scrolls=15):
    """Función más robusta para extraer posts"""
    posts = []
    
    try:
        driver.get(group_url)
        sleep(2)
        
        # Scroll para cargar contenido
        body = driver.find_element(By.TAG_NAME, 'body')
        for i in range(num_scrolls):
            body.send_keys(Keys.PAGE_DOWN)
            sleep(1)
        
        # Usar selectores más genéricos y robustos
        post_containers = driver.find_elements(By.CSS_SELECTOR, 
            "[data-ad-rendering-role='story_message'], div[dir='auto']")
        
        for container in post_containers:
            try:
                text = container.text.strip()
                if text and len(text) > 10:  # Filtrar textos muy cortos
                    posts.append(text)
            except Exception as e:
                continue
                
        return posts
        
    except Exception as e:
        print(f"Error extrayendo posts: {e}")
        return []

In [5]:
def extract_posts_improved(driver, group_url, num_scrolls=15):
    """
    Función mejorada para extraer posts de Facebook usando selectores CSS más específicos
    """
    posts_data = []
    
    driver.get(group_url)
    sleep(2)
    
    # Hacer scroll para cargar más contenido
    body = driver.find_element(By.TAG_NAME, 'body')
    for i in range(num_scrolls):
        body.send_keys(Keys.PAGE_DOWN)
        sleep(1)
        print(f"Scroll {i+1}/{num_scrolls} completado")
    
    # Buscar elementos con la clase específica que contiene los posts
    # Basado en el análisis del HTML, los posts están en spans con esta clase
    post_containers = driver.find_elements(By.CSS_SELECTOR, 
        "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u.x1yc453h")
    
    print(f"Encontrados {len(post_containers)} contenedores de posts")
    
    for i, container in enumerate(post_containers):
        try:
            # Buscar el texto dentro del contenedor
            text_elements = container.find_elements(By.CSS_SELECTOR, 'div[dir="auto"][style="text-align: start;"]')
            
            for text_element in text_elements:
                text_content = text_element.text.strip()
                if text_content and len(text_content) > 10:  # Filtrar textos muy cortos
                    posts_data.append({
                        'text': text_content,
                        'group_url': group_url,
                        'extraction_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'container_index': i
                    })
                    print(f"Post extraído ({len(text_content)} caracteres): {text_content[:100]}...")
            
            # Intentar hacer clic en "Ver más" si existe
            try:
                see_more_btn = container.find_element(By.CSS_SELECTOR, 
                    "div.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr")
                see_more_btn.click()
                sleep(1)
                
                # Volver a extraer el texto expandido
                expanded_text_elements = container.find_elements(By.CSS_SELECTOR, 'div[dir="auto"][style="text-align: start;"]')
                for text_element in expanded_text_elements:
                    expanded_text = text_element.text.strip()
                    if expanded_text and len(expanded_text) > len(text_content):  # Solo si es más largo
                        # Actualizar el último post agregado
                        if posts_data and posts_data[-1]['container_index'] == i:
                            posts_data[-1]['text'] = expanded_text
                            print(f"Texto expandido actualizado: {expanded_text[:100]}...")
                        
            except:
                pass  # No hay botón "Ver más" o no se pudo hacer clic
                
        except Exception as e:
            print(f"Error procesando contenedor {i}: {str(e)}")
            continue
    
    return posts_data

In [ ]:
# Probar la función mejorada con los grupos de Facebook
from datetime import datetime

groups_to_scrape = [
    "https://www.facebook.com/groups/973136421423238/"
    # Agregar más grupos aquí si es necesario
]

all_posts = []

for group_url in groups_to_scrape:
    print(f"\n=== Extrayendo posts de: {group_url} ===")
    posts = extract_posts_improved(driver, group_url, num_scrolls=15)
    all_posts.extend(posts)
    print(f"Total de posts extraídos de este grupo: {len(posts)}")

print(f"\n=== RESUMEN FINAL ===")
print(f"Total de posts extraídos de todos los grupos: {len(all_posts)}")

# Mostrar algunos ejemplos
for i, post in enumerate(all_posts[:3]):  # Mostrar los primeros 3 posts
    print(f"\nPost {i+1}:")
    print(f"Texto: {post['text'][:200]}..." if len(post['text']) > 200 else f"Texto: {post['text']}")
    print(f"Grupo: {post['group_url']}")
    print(f"Tiempo de extracción: {post['extraction_time']}")

In [52]:
def extract_posts_alternative_selectors(driver, group_url, num_scrolls=15):
    """
    Función alternativa que prueba diferentes selectores CSS para encontrar posts
    """
    posts_data = []
    
    driver.get(group_url)
    sleep(2)
    
    # Hacer scroll
    body = driver.find_element(By.TAG_NAME, 'body')
    for i in range(num_scrolls):
        body.send_keys(Keys.PAGE_DOWN)
        sleep(1)
    
    # Lista de selectores alternativos para probar
    selectors_to_try = [
        # Selector original del notebook
        "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u.x1yc453h",
        
        # Selectores más generales
        "div[dir='auto'][style*='text-align: start']",
        "div[dir='auto']",
        "span[dir='auto']",
        
        # Selectores basados en roles de datos
        "div[data-ad-rendering-role='story_message']",
        "div[data-ad-comet-preview='message']",
        
        # Selectores para contenido de posts
        "div.x1iorvi4.xjkvuk6.x1g0dm76.xpdmqnj",
        "div.xu06os2.x1ok221b"
    ]
    
    for selector in selectors_to_try:
        print(f"\nProbando selector: {selector}")
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            print(f"Encontrados {len(elements)} elementos")
            
            for i, element in enumerate(elements[:5]):  # Limitar a 5 elementos por selector
                try:
                    text = element.text.strip()
                    if text and len(text) > 20:  # Filtrar textos muy cortos
                        posts_data.append({
                            'text': text,
                            'selector_used': selector,
                            'group_url': group_url,
                            'element_index': i
                        })
                        print(f"  Texto encontrado: {text[:100]}...")
                except Exception as e:
                    continue
                    
        except Exception as e:
            print(f"Error con selector {selector}: {str(e)}")
            continue
    
    return posts_data

# Probar los selectores alternativos
print("=== PROBANDO SELECTORES ALTERNATIVOS ===")
alternative_posts = extract_posts_alternative_selectors(driver, 
    "https://www.facebook.com/groups/973136421423238/", 
    num_scrolls=5)  # Menos scrolls para prueba rápida

print(f"\nTotal de posts encontrados con selectores alternativos: {len(alternative_posts)}")

=== PROBANDO SELECTORES ALTERNATIVOS ===

Probando selector: span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u.x1yc453h
Encontrados 2 elementos
  Texto encontrado: Se vende bicicleta gw zebra r29
Precio: 650k negociables
Info:3017949343...
  Texto encontrado: Buenas noches, busco información sobre arriendos que queden cerca a la UIS, me serviría mucho un apa...

Probando selector: div[dir='auto'][style*='text-align: start']
Encontrados 4 elementos
  Texto encontrado: Se vende bicicleta gw zebra r29...
  Texto encontrado: Precio: 650k negociables...
  Texto encontrado: Buenas noches, busco información sobre arriendos que queden cerca a la UIS, me serviría mucho un apa...

Probando selector: div[dir='auto']
Encontrados 8 elementos
  Texto encontrado: Este es un grupo abierto para estudiantes, egresados, docentes y cualquier persona de la comunidad u...
  Texto encontrado: Se vende 

In [14]:

groups_links_list = [
        "https://www.facebook.com/photo?fbid=1274553160906661&set=gm.1048786260524920&idorvanity=973136421423238"
    ] #acá pegué todos los grupos de facebook uis que encontré


for group in groups_links_list:
    driver.get(group)
    sleep(2)
    body = driver.find_element(By.TAG_NAME, 'body')
    for _ in range(9):  #número de scrolls que se hacen por página
        body.send_keys(Keys.PAGE_DOWN)
        sleep(1)

        wait = WebDriverWait(driver, 15)
        comentarios = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[aria-label^="Comentario de"]')))
        for comentario in comentarios:
            print("Texto del comentario:")
            print(comentario.text)
            print("Clase real:")
            print(comentario.get_attribute("class"))

# Cerrar el navegador al finalizar  

Texto del comentario:
Frank Santamaria
En el parqueadero del estadio hay mucho espacio si ese es el problema (además de ser el lugar designado para que los estudiantes dejen sus vehículos), y si esas personas que "cuidan" los vehículos no tienen un trabajo digno, es porque saben que tienen… Ver más
2 sem
Me gusta
Responder
Compartir
48
Clase real:
x1n2onr6 x1g0dm76 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz
Texto del comentario:
Participante anónimo 565
Ese sector se estaba volviendo un chirreteadero, siempre habían personas fumando que ya parecía una olla. Una vez vi cómo dos personas de calle se estaban peleando con machetes en esa misma zona. Usted quiere su espacio para parquear porque es floja, y… Ver más
2 sem
Me gusta
Responder
Compartir
38
Clase real:
x1n2onr6 x1g0dm76 x1iorvi4 x78zum5 x1q0g3np x1a2a7pz
Texto del comentario:
Arias Oto
Pues ese sector ya se volvió un desorden completo, demasiados vehículos estacionados en esos alrededores, ya es imposible pasar por ahí.
2 sem
Me gusta
R

In [ ]:
print(driver.page_source)  # Esto te imprime todo el HTML visible para Selenium

<html id="facebook" class="_9dls __fb-light-mode" lang="es" dir="ltr"><head><link data-default-icon="https://static.xx.fbcdn.net/rsrc.php/y1/r/ay1hV6OlegS.ico" data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/UJj0tgk-RrT.ico" rel="shortcut icon" href="https://static.xx.fbcdn.net/rsrc.php/y1/r/ay1hV6OlegS.ico"><title>(7) Facebook</title><meta name="bingbot" content="noarchive"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no"><link rel="manifest" id="MANIFEST_LINK" href="/data/manifest/?is_workplace_mobile_pwa_dogfooding=0" crossorigin="use-credentials"><meta name="application-title" content="Facebook"><meta name="color-scheme" content="light"><meta name="theme-color" content="#FFFFFF"><style data-btmanifest="1023143320_main" nonce="">:root,.__fb-light-mode:root,.__fb-light-mode{--fds-black:#000000;--fds-black-alpha-05:rgba(0, 0, 0, .05);--fds-black-alpha-10:rgba(0, 0, 0, .1);--fds-black-alpha-15:rgba(0, 0, 0, .15);--fds-black

In [ ]:
driver.quit() #cerrar driver

Revisar Meta, llama, notebook. 